In [1]:
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import sleep, time
from random import randint
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import string
import string


In [2]:
##These labels are used to create the webpage of the yelp restaurants

In [128]:
website = "https://www.yelp.com/search?find_desc=Restaurants"
sf_label = "&find_loc=San%20Francisco%2C%20CA"
sj_label = "&find_loc=San%20Jose%2C%20CA"
fremont_label = "&find_loc=Fremont%2C%20CA"
la_label = "&find_loc=Los%20Angeles%2C%20CA"
ny_label = "&find_loc=New%20York%2C%20NY"
miami_label = "&find_loc=Miami%2C%20FL"
sac_label = "&find_loc=Sacramento%2C%20CA"
portland_label = "&find_loc=Portland%2C%20OR"
seattle_label = "&find_loc=Seattle%2C%20WA"
reno_label = "&find_loc=Reno%2C%20NV"
lv_label = "&find_loc=Las%20Vegas%2C%20NV"
phoenix_label = "&find_loc=Phoenix%2C%20AZ"
denver_label = "&find_loc=Denver%2C%20CO"
chicago_label = "&find_loc=Chicago%2C%20IL"
cheap_label = "&attrs=RestaurantsPriceRange2.1"
medium_label = "&attrs=RestaurantsPriceRange2.2"
expensive_label = "&attrs=RestaurantsPriceRange2.3"
vegetarian_label ="&cflt=vegetarian"
price_labels = ("&attrs=RestaurantsPriceRange2.1","&attrs=RestaurantsPriceRange2.2","&attrs=RestaurantsPriceRange2.3")
city_labels = ("&find_loc=San%20Francisco%2C%20CA","&find_loc=San%20Jose%2C%20CA","&find_loc=Fremont%2C%20CA","&find_loc=Los%20Angeles%2C%20CA","&find_loc=New%20York%2C%20NY","&find_loc=Miami%2C%20FL","&find_loc=Sacramento%2C%20CA","&find_loc=Portland%2C%20OR","&find_loc=Seattle%2C%20WA", "&find_loc=Reno%2C%20NV", "&find_loc=Las%20Vegas%2C%20NV", "&find_loc=Phoenix%2C%20AZ", "&find_loc=Denver%2C%20CO", "&find_loc=Chicago%2C%20IL")
pages = (1,2,3,4)

In [4]:
## The format is:
##(Website + Location_label + Price_label + page_Label) <nonveg
##(Website + location_label + price_label + vegetarian+label) < veg
## We cannot use a loop to scrape all the categories at once because certain categories have many more restaurants than others
## therefore, you have to just run this code procedurally for each tag.
## also, some pages have random advertisement restaurants that we have to get rid of as well
## If we insist on using a loop we have to decided exactly what locations/how many of each restaurant we are using

In [5]:

def getPage(inp):
    return ((inp-1)*30)

In [6]:
##insert page number as an arg. here
page = getPage(1)
page_label = "&start=" + str(page)
getPage(2)

30

In [7]:
##making as a list so we dont have to refactor as much in case we decide to change back to using a list 

toScrape = []
newSite = get(website+sf_label+cheap_label+page_label)
toScrape.append(newSite)
html_soup = BeautifulSoup(newSite.text, 'html.parser')

In [129]:

def veg_links_scraper(link):
    response = get(link)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    refList = []
    yelp_template = string.Template("https://www.yelp.com$link")
    restaurant_containers = html_soup.find_all('div', class_ = 'lemon--div__373c0__1mboc searchResult__373c0__1yggB border-color--default__373c0__2oFDT')
    for current in restaurant_containers:
        title = current.h3.find('a', class_ = "lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5")
        refList.append(title['href'])
       # print(title['href'])
    for i in range(len(refList)):
        refList[i] = yelp_template.substitute(link = refList[i])

    for i in refList:
        if("ad_business" in i): #Gets rid of the advertisement
            refList.remove(i)

    
    #print(refList)
    return refList
#removing the first restaurant bc its an advertised restaurant I think


In [130]:
#https://www.yelp.com/search?find_desc=Restaurants&find_loc=San%20Francisco%2C%20CA&attrs=RestaurantsPriceRange2.1&start=30 2nd page 
#https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los%20Angeles%2C%20CA&attrs=RestaurantsPriceRange2.1&cflt=vegetarian&start=30 Veg 2nd page
#https://www.yelp.com/search?find_desc=Restaurants&find_loc=San%20Francisco%2C%20CA&attrs=RestaurantsPriceRange2.1&cflt=vegetarian&start=0
veg_links = []
cash_dict = {}
def scrape_non_veg_links():
    link_template = string.Template("https://www.yelp.com/search?find_desc=Restaurantss&find_loc=San%20Francisco%2C%20CA$price&start=$pageNum")
    for label in price_labels:
        for page in pages:
            non_veg_links.append(link_template.substitute(price = label, pageNum = getPage(page)))
            
def scrape_veg_links(veg_links):
    temp_list = []
    link_template = string.Template("https://www.yelp.com/search?find_desc=Restaurants$cityLabel$price$veg&start=$pageNum")
    for city in city_labels:
        for label in price_labels:
            
            for page in pages:
                temp_list = veg_links_scraper(link_template.substitute(cityLabel = city, price = label,veg = vegetarian_label, pageNum = getPage(page)))
                print(temp_list)
               #veg_links.append(link_template.substitute(price = label,veg = vegetarian_label, pageNum = getPage(page)))
                if label in cash_dict:
                    cash_dict[label] += len(temp_list)
                    
                else:
                    cash_dict[label] = len(temp_list)
                    
                #veg_links.extend(veg_links_scraper(link_template.substitute(cityLabel = city, price = label,veg = vegetarian_label, pageNum = getPage(page))))
                
                veg_links.extend(temp_list)
        print (cash_dict)
        print(city)
        print(len(veg_links)) 
        
    return veg_links
scrape_non_veg_links()
real_veg_links = scrape_veg_links(veg_links)


['https://www.yelp.com/biz/the-dosa-brothers-san-francisco-10?osq=Restaurants', 'https://www.yelp.com/biz/peasant-pies-noe-valley-san-francisco?osq=Restaurants', 'https://www.yelp.com/biz/lucky-creation-vegetarian-restaurant-san-francisco?osq=Restaurants', 'https://www.yelp.com/biz/mortys-delicatessen-san-francisco?osq=Restaurants', 'https://www.yelp.com/biz/halal-shop-san-francisco-2?osq=Restaurants', 'https://www.yelp.com/biz/peasant-pies-caf%C3%A9-and-catering-san-francisco-5?osq=Restaurants', 'https://www.yelp.com/biz/peasant-pies-caf%C3%A9-and-catering-san-francisco-2?osq=Restaurants', 'https://www.yelp.com/biz/something-simple-for-lunch-san-francisco?osq=Restaurants', 'https://www.yelp.com/biz/cafe-insalata-san-francisco?osq=Restaurants', 'https://www.yelp.com/biz/the-dosa-brothers-san-francisco-12?osq=Restaurants', 'https://www.yelp.com/biz/bean-sprouts-sausalito-2?osq=Restaurants']
[]
[]
[]
[]
[]
[]
[]
['https://www.yelp.com/biz/greens-restaurant-san-francisco-5?osq=Restaurants

[]
[]
['https://www.yelp.com/biz/pecorino-restaurant-los-angeles?osq=Restaurants', 'https://www.yelp.com/biz/plant-food-wine-venice?osq=Restaurants']
[]
[]
[]
{'&attrs=RestaurantsPriceRange2.1': 25, '&attrs=RestaurantsPriceRange2.2': 85, '&attrs=RestaurantsPriceRange2.3': 8}
&find_loc=Los%20Angeles%2C%20CA
118
['https://www.yelp.com/biz/superiority-burger-new-york?osq=Restaurants', 'https://www.yelp.com/biz/ny-dosas-new-york?osq=Restaurants', 'https://www.yelp.com/biz/govindas-vegetarian-brooklyn?osq=Restaurants', 'https://www.yelp.com/biz/taim-west-village-new-york?osq=Restaurants', 'https://www.yelp.com/biz/newtown-brooklyn-2?osq=Restaurants', 'https://www.yelp.com/biz/burrito-republic-ridgewood?osq=Restaurants', 'https://www.yelp.com/biz/dil-e-punjab-deli-new-york-2?osq=Restaurants', 'https://www.yelp.com/biz/chick-p-brooklyn?osq=Restaurants', 'https://www.yelp.com/biz/doaba-deli-new-york-2?osq=Restaurants', 'https://www.yelp.com/biz/green-symphony-new-york-2?osq=Restaurants', 'http

['https://www.yelp.com/biz/dirt-candy-new-york-2?osq=Restaurants', 'https://www.yelp.com/biz/osteria-57-new-york?osq=Restaurants', 'https://www.yelp.com/biz/blossom-new-york-3?osq=Restaurants', 'https://www.yelp.com/biz/hangawi-new-york?osq=Restaurants', 'https://www.yelp.com/biz/vatan-indian-vegetarian-new-york-2?osq=Restaurants', 'https://www.yelp.com/biz/avant-garden-new-york?osq=Restaurants', 'https://www.yelp.com/biz/the-garden-new-york?osq=Restaurants']
[]
[]
[]
{'&attrs=RestaurantsPriceRange2.1': 55, '&attrs=RestaurantsPriceRange2.2': 175, '&attrs=RestaurantsPriceRange2.3': 15}
&find_loc=New%20York%2C%20NY
245
['https://www.yelp.com/biz/amsterdam-falafelshop-south-miami?osq=Restaurants', 'https://www.yelp.com/biz/la-latina-miami?osq=Restaurants', 'https://www.yelp.com/biz/harvest-moon-gourmet-bistro-miami-springs?osq=Restaurants', 'https://www.yelp.com/biz/freshii-miami?osq=Restaurants', 'https://www.yelp.com/biz/deezfruta-juice-and-smoothies-miami-2?osq=Restaurants']
[]
[]
[]
[

[]
[]
[]
[]
['https://www.yelp.com/biz/thali-reno?osq=Restaurants', 'https://www.yelp.com/biz/laughing-planet-cafe-reno?osq=Restaurants', 'https://www.yelp.com/biz/bab-caf%C3%A9-reno-reno-5?osq=Restaurants', 'https://www.yelp.com/biz/mayas-south-indian-cuisine-reno-2?osq=Restaurants', 'https://www.yelp.com/biz/buenos-grill-reno?osq=Restaurants', 'https://www.yelp.com/biz/sushi-lover-reno?osq=Restaurants', 'https://www.yelp.com/biz/the-blind-onion-pizza-and-pub-reno?osq=Restaurants', 'https://www.yelp.com/biz/ikes-sandwiches-reno?osq=Restaurants']
[]
[]
[]
[]
[]
[]
[]
{'&attrs=RestaurantsPriceRange2.1': 88, '&attrs=RestaurantsPriceRange2.2': 243, '&attrs=RestaurantsPriceRange2.3': 19}
&find_loc=Reno%2C%20NV
350
['https://www.yelp.com/biz/parsley-modern-mediterranean-las-vegas-3?osq=Restaurants', 'https://www.yelp.com/biz/parsley-modern-mediterranean-las-vegas-2?osq=Restaurants', 'https://www.yelp.com/biz/india-market-and-cafe-las-vegas?osq=Restaurants', 'https://www.yelp.com/biz/sunrise

['https://www.yelp.com/biz/el-pecado-chicago?osq=Restaurants', 'https://www.yelp.com/biz/greenhouse-inn-chicago-2?osq=Restaurants', 'https://www.yelp.com/biz/freshii-chicago-6?osq=Restaurants', 'https://www.yelp.com/biz/farmers-fridge-chicago-14?osq=Restaurants', 'https://www.yelp.com/biz/freshii-chicago-33?osq=Restaurants', 'https://www.yelp.com/biz/freshii-chicago-9?osq=Restaurants', 'https://www.yelp.com/biz/freshii-chicago-21?osq=Restaurants', 'https://www.yelp.com/biz/freshii-chicago-23?osq=Restaurants', 'https://www.yelp.com/biz/d-thai-lyons?osq=Restaurants', 'https://www.yelp.com/biz/freshii-chicago-13?osq=Restaurants', 'https://www.yelp.com/biz/freshii-chicago-17?osq=Restaurants']
[]
[]
[]
[]
[]
[]
{'&attrs=RestaurantsPriceRange2.1': 133, '&attrs=RestaurantsPriceRange2.2': 321, '&attrs=RestaurantsPriceRange2.3': 19}
&find_loc=Chicago%2C%20IL
473
